In [1]:
%cd hypernet

/home/z1157095/hypernet-cnn/hypernet


In [2]:
from dotenv import load_dotenv
load_dotenv()

import random
import os

In [3]:
from comet_ml import Experiment, Optimizer

import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
import torch.utils.data as data_utils
import pandas as pd
from collections import defaultdict

torch.set_default_dtype(torch.float32)

In [4]:
from torchsummary import summary
import matplotlib.pyplot as plt
from tqdm import tqdm, trange

In [5]:
import tabular_hypernet as hp
from tabular_hypernet.semisl import SSLCELossWithThreshold

In [6]:
hp.training_utils.get_dataset

<function tabular_hypernet.training_utils.get_dataset(size=60000, masked=False, mask_no=200, mask_size=700, shared_mask=False, batch_size=32, test_batch_size=32)>

In [7]:
os.environ['COMET_KEY'] = 'UXrV5UxyhTK3cyQNG6BDuc4bE'
os.environ.get("COMET_KEY")

'UXrV5UxyhTK3cyQNG6BDuc4bE'

### Setup for training

In [8]:
seed = 5

In [9]:
np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)

In [13]:
mask_size = 100
masks_no = 100

In [10]:
dataset = hp.semisl.get_train_test_sets()

In [11]:
epochs = 100



results = defaultdict(list)
size = (100, 59900)

In [12]:
#68.6   1.78

### Train 

In [13]:
for masks_no in [50, 200]:
    for mask_size in [50, 200]:
        for beta in [.1, 1.]:
            for loss_threshold in [0.8]:
                for lr in [3e-5]:


                    criterion = SSLCELossWithThreshold(beta=beta, threshold=loss_threshold)

                    np.random.seed(seed)
                    torch.manual_seed(seed)
                    random.seed(seed)

                    # dataset


                    hypernet = hp.Hypernetwork(
                        architecture=torch.nn.Sequential(
                            torch.nn.Linear(784, 64), 
                            torch.nn.ReLU(),
                            torch.nn.Linear(64, 256),
                            torch.nn.ReLU(),
                            torch.nn.Linear(256, 128),
                        ),
                        target_architecture=[(mask_size, 100), (100, 10)],
                        test_nodes=masks_no,
                    ).cuda()


                    hypernet = hypernet.train()



                    optimizer = torch.optim.Adam(hypernet.parameters(), lr=lr)

                    # loaders
                    sup_trainloader, unsup_trainloader, testloader = hp.semisl.get_dataloaders(dataset=dataset, size=size, batch_size=32, test_batch_size=64)
                    trainloader = hp.semisl.TrainDataLoaderSemi(sup_trainloader, unsup_trainloader)

                    results[size].append(hp.semisl.train_semisl(hypernet,
                                                      optimizer,
                                                      criterion,
                                                      (trainloader, testloader), 
                                                      size,
                                                      epochs,
                                                      masks_no,
                                                      changing_beta=None,
                                                      log_to_comet=True,
                                                      tags=['mask size & no playing'],
                                                      description="""
                                                      trying different masks no and mask size to test in ssl setting
                                                      """,
                                                    log_params={'seed': seed, 'temp scheduler': 'none', 'shuffled masks': 'true'}
                                                    ))


torch.Size([1, 128])


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


!! log to comet is True 



COMET INFO: Experiment is live on comet.ml https://www.comet.com/abulenok/semi-hypernetwork/7f27bf5f70a74c4fbe5a1492f73e48f5

100%|██████████████████████████████████████████████████████████████████████████| 100/100 [33:59<00:00, 20.39s/it, loss=1.79, test_acc=71]
COMET ERROR: Error sending a notification, make sure you have opted-in for notifications
COMET INFO: Uploading metrics, params, and assets to Comet before program termination (may take several seconds)
COMET INFO: The Python SDK has 3600 seconds to finish before aborting...
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


torch.Size([1, 128])
!! log to comet is True 



...

COMET INFO: Experiment is live on comet.ml https://www.comet.com/abulenok/semi-hypernetwork/82fc08b32d6f4dc0a8c7f2534cea77aa

100%|████████████████████████████████████████████████████████████████████████| 100/100 [36:34<00:00, 21.95s/it, loss=1.81, test_acc=70.9]
COMET ERROR: Error sending a notification, make sure you have opted-in for notifications
COMET INFO: Uploading 1 metrics, params and output messages


torch.Size([1, 128])
!! log to comet is True 



COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.com/abulenok/semi-hypernetwork/c0f776307b40451d9b7ce2078d83ebec

100%|██████████████████████████████████████████████████████████████████████████| 100/100 [44:24<00:00, 26.65s/it, loss=1.72, test_acc=76]
COMET ERROR: Error sending a notification, make sure you have opted-in for notifications
COMET INFO: Uploading 1 metrics, params and output messages


torch.Size([1, 128])
!! log to comet is True 



COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.com/abulenok/semi-hypernetwork/8e5cefd0cdea4d3cb99b1452c442dbfe

100%|████████████████████████████████████████████████████████████████████████| 100/100 [48:49<00:00, 29.30s/it, loss=1.73, test_acc=74.8]
COMET ERROR: Error sending a notification, make sure you have opted-in for notifications
COMET INFO: Uploading 1 metrics, params and output messages
